In [36]:
import cv2 
import glob
import numpy as np

In [37]:
samples=[]
targets=[]

In [38]:
def list_to_array(samples,targets):
    samples=np.array(samples)
    targets=np.array(targets)
    return samples,targets

In [39]:
def prepare_data(image,data):
    # convert to grayscale, and find edges
    
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_OTSU + cv2.THRESH_BINARY)[1]

    # Find contour and sort by contour area
    cnts = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    cnts = sorted(cnts, key=cv2.contourArea, reverse=True)

    # Find bounding box and extract ROI
    for c in cnts:
        x,y,w,h = cv2.boundingRect(c)
        ROI = image[y:y+h, x:x+w]
        
        ROI = cv2.resize(ROI, (400, 400),interpolation = cv2.INTER_CUBIC)
        samples.append(ROI)
        if data=='yes':
            targets.append(1)
        else:
            targets.append(0)
        break





In [40]:
# Load images (yes)
for img in glob.glob("yes/*.jpg"):
    image= cv2.imread(img)
    prepare_data(image,'yes')
    

In [41]:
# Load images (no)
for img in glob.glob("no/*.jpg"):
    image= cv2.imread(img)
    prepare_data(image,'no')
# convert lists of data to np array
else:
    samples,targets= list_to_array(samples,targets)
    
    

In [42]:
samples.shape

(103, 400, 400, 3)

In [43]:
# 2D array
samples=samples.reshape(103,480000)

In [44]:
# split data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(samples,targets, test_size=0.15, random_state=42)

In [45]:
# train a model for test without any optimization
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

pipe = Pipeline([ ('scaler', StandardScaler()),('svc', SVC())])
pipe.fit(X_train, y_train)
y_predict=pipe.predict(X_test)
accuracy_score(y_test,y_predict)



0.75

طبق انتظار عدد بسیار پایینی می باشد از علل می توان به تعداد کم نمونه و همچنین تعداد بسیار زیاد فیچر ها در قیاس با تعداد نمونه ها نام برد


به همین علت نیاز به feature extraction کاملا حس می شود

#### feature extraction

In [46]:
from sklearn.decomposition import PCA
def f_extraction(percentage,data):
    pca = PCA(n_components=percentage)
    X_train_reduced= pca.fit_transform(data)
    # number of features
    n_features=pca.n_components_

    return pca,X_train_reduced,n_features

In [47]:
pca,X_train_reduced,n_features=f_extraction(0.95,X_train)
print(n_features)

56


باتوجه به اینکه عدد ۵۶ در قیاس با تعداد نمونه ها مقداری زیاد است سعی در کاهش بیشتر داریم

In [63]:
pca2,X_train_reduced2,n_features=f_extraction(0.70,X_train)
print(n_features)

20


ابتدا با ۵۶ فیچر تست می کنیم

In [49]:
pipe.fit(X_train_reduced, y_train)
X_test_reduced=pca.transform(X_test)
y_predict2=pipe.predict(X_test_reduced)
accuracy_score(y_test,y_predict2)

0.8125

با ۲۰ فیچر

In [64]:
pipe.fit(X_train_reduced2, y_train)
X_test_reduced2=pca2.transform(X_test)
y_predict3=pipe.predict(X_test_reduced2)
accuracy_score(y_test,y_predict3)

0.75

برخلاف انتظار با ۵۶ فیچر جواب بهتری بدست امده 
اما نکته مهم ان است که تنها با ۲۰ فیچر می توان به ۰.۷۵ رسید که خوب است

#### Xgboost


In [79]:
import xgboost as xgb


In [92]:
def xgb_d (X_train,X_test)  :  
    
    
    xgb_0 = xgb.XGBClassifier()
    
    xgb_0.fit(X_train,y_train)
    Y_predictx=xgb_0.predict(X_test)
    return accuracy_score(y_test, Y_predictx)

In [94]:
xgb_d(X_train_reduced2,X_test_reduced2)

0.875

In [93]:
xgb_d(X_train,X_test)

0.8125

در این مرحله تعداد فیچر های ۲۰ توانست بالاترین میزان درست بودن جواب را بدهد و از نظر زمانی بسیار سریعتر به نسبت اینکه تمام فیچر ها حضور داشته باشن عمل کند

استفاده از gridsearch برای پیدا کردن بهترین ۱ارامتر های 

#### tunning xgboost

In [106]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    "max_depth": [3, 4, 5, 7],
    "learning_rate": [0.1, 0.01, 0.05,0.2],
    "gamma": [0, 0.25, 1,3],
    "reg_lambda": [0, 1, 10],
    "scale_pos_weight": [1, 3, 5],
    "subsample": [0.8],
    "colsample_bytree": [0.5],
}

In [107]:
xgb_tun = xgb.XGBClassifier(objective="binary:logistic")
grid_cv = GridSearchCV(xgb_tun, param_grid, n_jobs=-1, cv=3, scoring="roc_auc")
model = grid_cv.fit(X_train_reduced2, y_train)
grid_cv.best_score_

0.7778718865675387

In [110]:
y_predict_xg_tun= model.predict(X_test_reduced2)
accuracy_score(y_test, y_predict_xg_tun)

0.75